## Introduction
Welcome to this script! My name is GPT-4, a large language model developed by OpenAI. I was designed with the ability to understand and generate human-like text based on the information I was trained on.

This script was written by me with human help with the intention of facilitating better communication between humans and myself. It leverages my ability to process and summarize large volumes of text, making it easy to extract the core ideas from a detailed document.

In [1]:
import summarize
from secret_key import OPENAI_API_KEY

### Define initial variables

In [6]:
# Where is your pdf?
file_path = 'path/to/your/pdf'
file_path = 'pdfs/ifc_reports/508682-1001-REP-JJ-0001 T1 100%.pdf'

# Remember to write a custom message that explains what you are sending to GPT, and what you want it to do!
custom_message_file = "custom_message.txt"

# Set GPT version (GPT-4 is way better), uncomment accordingly
gpt_version = 'gpt-3.5-turbo'
#gpt_version = 'gpt-4'

# Set limit (so you don't accidentally lose all your money)
max_api_calls = 50

### Run this cell to start the summarization (upto 20 minutes for long documents)

In [7]:
text = summarize.pdf_to_text(file_path)
preprocessed_text = summarize.preprocess(text)
text_chunks = summarize.create_text_chunks(preprocessed_text)
custom_message = summarize.read_custom_message(custom_message_file)

api_calls = 0
summaries = []

summaries = summarize.generate_summaries(
    text_chunks,
    custom_message,
    max_api_calls,
    gpt_version,
    OPENAI_API_KEY
)

summarize.save_summaries_to_json(summaries)
summarize.format_and_save_summaries()